In [28]:
from __future__ import absolute_import
from __future__ import print_function
import scipy.io
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from six.moves import range
import keras.utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
import os
import sys
import json
import io
from keras.utils import to_categorical
from keras.models import Model
from keras.initializers import Constant
import os.path
from os import path
sloc = '../input/coastal_city_by_day/'
ddir = '../../Second Twitter Flooding Paper/Data/Daily Tweets Coastal Cities/'
flist = os.listdir(ddir)
from itertools import compress
exists = [path.exists(sloc + f)==False for f in flist]
flist = list(compress(flist, exists))

In [29]:
with open('../model_objects/tokenizer_V2.json') as f: 
        data = json.load(f) 
        tokenizer = tokenizer_from_json(data)
MAX_SEQUENCE_LENGTH = 100

new_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(new_word_index))
        
# load json and create model
json_file = open('../model_objects/flood_classifier_model_V2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../model_objects/flood_classifer_model_V2.h5")
print("Loaded model from disk")



Found 26251 unique tokens.
Loaded model from disk


In [30]:
def f_predict_values(file):
    ddir = '../../Second Twitter Flooding Paper/Data/Daily Tweets Coastal Cities/'
    new_name = file
    fname =  ddir + file
    #print(fname)
    df = pd.read_csv(fname, engine='python')
    df = df[df['text'].notna()]
    full_texts = [t for t in df['text']]
    sequences = tokenizer.texts_to_sequences(full_texts)
    new_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    preds = loaded_model.predict(new_data)
    df['Probability_Flooding_V2'] = preds
    df.to_csv('../input/coastal_city_by_day/'+os.path.basename(fname))

In [31]:
flist

['2019_12_11.csv', '2019_12_12.csv', '2019_12_13.csv', '2019_12_14.csv']

In [ ]:
for f in flist:
    f_predict_values(f)

In [16]:
import time
from multiprocessing import Pool
import workers
if __name__ ==  '__main__': 
    num_processors = 4
    p=Pool(processes = num_processors)
    output = p.map(workers.worker,flist[:20])
print(output)

SyntaxError: invalid syntax (workerpool.py, line 173)